In [23]:
import pandas as pd
import ipaddress

# Read the excel file : 
df = pd.read_excel('ip_data.xlsx')
print(df)


       IP Address    Subnet Mask
0    192.168.1.24  255.255.255.0
1        10.1.5.4  255.255.254.0
2    172.16.20.33  255.255.255.0
3   192.168.100.7  255.255.252.0
4      10.0.3.112  255.255.255.0
5    172.16.1.200  255.255.254.0
6       10.2.1.56  255.255.252.0
7     192.168.2.3  255.255.255.0
8     172.16.50.1  255.255.252.0
9       10.10.0.5  255.255.254.0
10   192.168.3.14  255.255.255.0
11      10.20.4.6  255.255.252.0
12     172.16.8.9  255.255.254.0
13       10.4.3.2  255.255.255.0
14  192.168.20.44  255.255.252.0
15   172.16.40.22  255.255.255.0
16     10.0.0.200  255.255.254.0
17   192.168.10.1  255.255.255.0
18   172.16.15.15  255.255.254.0
19       10.3.3.9  255.255.252.0
20    192.168.4.5  255.255.255.0
21      10.50.2.7  255.255.255.0
22   172.16.60.30  255.255.252.0
23    192.168.7.8  255.255.254.0
24     10.15.5.50  255.255.252.0


In [25]:
def analyze_subnet(row):
    ip_str = row['IP Address']
    subnet_mask = row['Subnet Mask']
    net = ipaddress.IPv4Network(f"{ip_str}/{subnet_mask}", strict=False)
    return pd.Series({
        'CIDR': net.with_prefixlen,
        'Network': str(net.network_address),
        'Broadcast': str(net.broadcast_address),
        'UsableHosts': net.num_addresses - 2
    })

results = df.apply(analyze_subnet, axis=1)
df = pd.concat([df, results], axis=1)


In [26]:
# Which subnet has the most hosts ? 
subnet_summary = df.groupby('CIDR')['UsableHosts'].sum().reset_index()

most_hosts = df.loc[df['UsableHosts'].idxmax()]
print(most_hosts)

most_hosts_subnet = subnet_summary.loc[subnet_summary['UsableHosts'].idxmax()]
print(most_hosts_subnet)


IP Address        192.168.100.7
Subnet Mask       255.255.252.0
CIDR           192.168.100.0/22
Network           192.168.100.0
Broadcast       192.168.103.255
UsableHosts                1022
Name: 3, dtype: object
CIDR           10.15.4.0/22
UsableHosts            1022
Name: 4, dtype: object


In [27]:
# Are there any overlapping subnets ? 

all_networks = [ipaddress.IPv4Network(cidr) for cidr in df['CIDR']]
overlaps = []

for i, net1 in enumerate(all_networks):
    for j, net2 in enumerate(all_networks):
        if i != j and (net1.overlaps(net2)):
            overlaps.append((net1.with_prefixlen, net2.with_prefixlen))

print(overlaps)


[]


In [28]:
# What is the smallest and largest subnet ?

df['PrefixLength'] = df['CIDR'].apply(lambda x: int(x.split('/')[1]))
smallest = df.loc[df['PrefixLength'].idxmax()]  # smallest block = largest prefix length
largest  = df.loc[df['PrefixLength'].idxmin()]  # largest block = smallest prefix length
print("Smallest subnet:", smallest)
print("Largest subnet:", largest)


Smallest subnet: IP Address        192.168.1.24
Subnet Mask      255.255.255.0
CIDR            192.168.1.0/24
Network            192.168.1.0
Broadcast        192.168.1.255
UsableHosts                254
PrefixLength                24
Name: 0, dtype: object
Largest subnet: IP Address         192.168.100.7
Subnet Mask        255.255.252.0
CIDR            192.168.100.0/22
Network            192.168.100.0
Broadcast        192.168.103.255
UsableHosts                 1022
PrefixLength                  22
Name: 3, dtype: object
